# Transfer Attack Experiment

In this document we will investigate the accuracy of the A-suffix models on the adverarial examples generated from their sister B-suffix models.

In [23]:
import loadVim as lv
import torch
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
def compute_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1) 
    correct = torch.sum(preds == labels).item() 
    return correct / len(labels)

def validate(model, valid_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = torch.nn.CrossEntropyLoss()
    
    model.eval()
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.logits, labels)

            accuracy = compute_accuracy(outputs.logits, labels)
            total_accuracy += accuracy
            total_loss += loss.item()
    
    average_loss = total_loss / len(valid_loader)
    average_accuracy = total_accuracy / len(valid_loader)
    return average_loss, average_accuracy

# Vim Transfer Attack Successrate

In [29]:
# Load weights into model
model, processor = lv.prepareDownstreamVim()
pathcheckpoint = "/home/jupyter/AdversarialRobustness/models/VimA.pth"
checkpoint = torch.load(pathcheckpoint)
model.vim_model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

# Load adversarial data from B set
pathadvdata = "/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvVimB"

def preproc(path):
    tensor = torch.load(path)
    return tensor.squeeze(0).cpu()

datasetAdv = datasets.DatasetFolder(pathadvdata, loader = preproc, extensions=".pt")
loaderAdv = DataLoader(datasetAdv, batch_size=32, shuffle=False)

validate(model, loaderAdv)

(4.55005669593811, 0.12408088235294118)

# Vit Transfer Attack Successrate

In [31]:
# Load weights into model
model, processor = lv.prepareDownstreamVit()
pathcheckpoint = "/home/jupyter/AdversarialRobustness/models/VitA.pth"
checkpoint = torch.load(pathcheckpoint)
model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

# Load adversarial data from B set
pathadvdata = "/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvVitB"

def preproc(path):
    tensor = torch.load(path)
    return tensor.squeeze(0).cpu()

datasetAdv = datasets.DatasetFolder(pathadvdata, loader = preproc, extensions=".pt")
loaderAdv = DataLoader(datasetAdv, batch_size=32, shuffle=False)

validate(model, loaderAdv)

(3.2605673585619246, 0.20089285714285715)

# ResNet Transfer Attack Successrate

In [34]:
# Load weights into model
model, processor = lv.prepareDownstreamResnet()
pathcheckpoint = "/home/jupyter/AdversarialRobustness/models/ResNetA.pth"
checkpoint = torch.load(pathcheckpoint)
model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

# Load adversarial data from B set
pathadvdata = "/home/jupyter/AdversarialRobustness/CelebAdv/CelAdvResNetB"

def preproc(path):
    tensor = torch.load(path)
    return tensor.squeeze(0).cpu()

datasetAdv = datasets.DatasetFolder(pathadvdata, loader = preproc, extensions=".pt")
loaderAdv = DataLoader(datasetAdv, batch_size=32, shuffle=False)

validate(model, loaderAdv)

(1.7991121113300323, 0.3802083333333333)